In [1]:
%matplotlib inline
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
from typing import Optional, Tuple, Dict
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import pandas as pd
import os
from polyphys.manage.typer import ParserT
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCub
from polyphys.manage.organizer import invalid_keyword, sort_filenames
from polyphys.analyze import clusters
from polyphys.probe.prober import stamps_report
import warnings

In [3]:
from polyphys.manage.parser import HnsCub

In [4]:
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi_mini/research_data/hns_cubic-probe/N200epshm29nh37ac2/N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring"
nuc_pairs = glob(macbookpro_path + '/N*' + group + '*')
nuc_pairs = sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)

In [5]:
nuc_pairs

[('/Users/amirhsi_mini/research_data/hns_cubic-probe/N200epshm29nh37ac2/N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring/N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/hns_cubic-probe/N200epshm29nh37ac2/N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring/N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring.nucleoid.lammpstrj')]

In [6]:
topology = nuc_pairs[0][0]
trajectory = nuc_pairs[0][1]
lineage = 'whole'
save_to = './'
continuous = False
if (lineage == 'segment') & (continuous is False):
    warnings.warn(
        "lineage is "
        f"'{lineage}' "
        "and 'continuous' is "
        f"'{continuous}. "
        "Please ensure the "
        f"'{trajectory}' is NOT part of a sequence of trajectories.",
        UserWarning
    )
print("Setting the name of analyze file...")
sim_info = HnsCub(
    trajectory,
    lineage,
    'cubic',
    'nucleoid'
)
sim_name = sim_info.lineage_name + "-" + sim_info.group
print("\n" + sim_name + " is analyzing...\n")
# LJ time difference between two consecutive frames:
time_unit = sim_info.dmon * np.sqrt(
    sim_info.mmon * sim_info.eps_others)  # LJ time unit
lj_nstep = sim_info.ndump  # Sampling steps via dump command in Lammps
lj_dt = sim_info.dt
sim_real_dt = lj_nstep * lj_dt * time_unit
cell = mda.Universe(
    topology, trajectory, topology_format='DATA',
    format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
    atom_style="id resid type x y z", dt=sim_real_dt
    )
# slicing trajectory based the continuous condition
if continuous:
    sliced_trj = cell.trajectory[0: -1]
    n_frames = cell.trajectory.n_frames - 1
else:
    sliced_trj = cell.trajectory
    n_frames = cell.trajectory.n_frames
# selecting atom groups
bug = cell.select_atoms('resid 1')  # the bug
hns_hole = cell.select_atoms('type 2')  # the hns holes
hns_core = cell.select_atoms('type 3')  # the hns cores

Setting the name of analyze file...


/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/PolyPhys/polyphys/manage/parser.py:1940: UserWarning: It is assumed that 'nc' is the last attribute shortkey in a lineage_name of types: 'ensemble', 'ensemble_long', 'whole', 'segment'.
  warnings.warn(convention_warning, UserWarning)



N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring-nucleoid is analyzing...



In [36]:
def apply_pbc(
    pbc_lengths: np.ndarray,
    pbc_lengths_inverse: np.ndarray,
    pbc: dict
) -> Tuple[np.ndarray, np.ndarray]:
    """Updates the periodic boundary condition (PBC) information in the
    dimentions (keys) by the length (values) passed by `pbc`. The key
    are indexes of the `pbc_lengths` and `pbc_lengths_inverse` arrays.

    Parameters
    ----------
    pbc_lengths: numpy.ndarray
        An array of lengths/sizes in different directions.
    pbc_lengths_inverse: numpy.ndarray
        An array of the invesres of lengths/sizes in different directions.
    pbc: dict
        A dictionary of dimensions (keys) and lenghts (values) in which
        the pbc exists.

    Return
    ------
    pbc_lengths: numpy.ndarray
        The updated array of lengths/sizes in different directions.
    pbc_lengths_inverse: numpy.ndarray
        The updated array of the invesres of lengths/sizes in different
        directions.
    """
    for dim, length in pbc.items():
        pbc_lengths[dim] = length
        pbc_lengths_inverse[dim] = 1 / length
    return pbc_lengths, pbc_lengths_inverse


def dist_sq_matrix(
    positions: np.ndarray,
    pbc: Optional[dict] = None
) -> np.ndarray:
    """Finds the squares of distances between all the pair of atoms
    (even self-distances where i=j) for a given list of atoms' `positions`.

    Parameters
    ----------
    positions: np.ndarray
        a matrix with size n_atoms * n_dims of atoms' positions in which
        the rows are atoms' positions and the columns are different dimentions
        or components.

    pbc: dict, defualt None
        A dictionary of dimensions (keys) and lenghts (values) in which
        the pbc exists.

    Rutern
    ------
    dist_sq: np.ndarray
        A square matrix of size n_atoms * n_atoms in which each elements are
        the squares of the center-to-center distances between different pairs
        of atoms. All the diagonal elements of this materix is 0.

    References
    ----------
    2019 - Albanie S - Euclidean Distance Matrix Trick
    """
    n_atoms, n_dims = positions.shape
    # Defining pbc lengths with value zero in all directions:
    # The approach allows us to combine linear agebra and numpy broadcasting
    # and efficently apply pbc.
    pbc_lengths = np.zeros(n_dims)
    pbc_lengths_inverse = np.zeros(n_dims)
    if pbc is not None:
        pbc_lengths, pbc_lengths_inverse = apply_pbc(
            pbc_lengths, pbc_lengths_inverse, pbc
        )
    # Redefining the shapes of different arrays to combine linear algebra
    # with numpy broadcasting.
    pbc_lengths = np.reshape(pbc_lengths, (n_dims, 1, 1))
    pbc_lengths_inverse = np.reshape(pbc_lengths_inverse, (n_dims, 1, 1))
    pos_T = positions.T
    pos_j_element = np.reshape(pos_T, (n_dims, n_atoms, 1))
    pos_i_element = np.reshape(pos_T, (n_dims, 1, n_atoms))
    # differences:
    dist_sq = pos_j_element - pos_i_element
    # applying pbc
    dist_sq = dist_sq - pbc_lengths * np.around(pbc_lengths_inverse * dist_sq)
    print(dist_sq)
    # squaring each elements
    dist_sq = dist_sq ** 2
    # sum of axis=0 means sum over d x_ijd^2 where d is the number of
    # dimentions of a given r_ij^2
    dist_sq = np.sum(dist_sq, axis=0)
    return dist_sq

In [40]:
x4[:,0]/50

array([0.02, 0.08, 0.12])

In [37]:
x1 = 1* np.ones(3)
print(x1)
x2 = 4 * np.ones(3)
print(x2)
x3 = 6 * np.ones(3)
print(x3)
x4 = np.array([x1, x2, x3])#.reshape(2,3,1)
print(x4)

[1. 1. 1.]
[4. 4. 4.]
[6. 6. 6.]
[[1. 1. 1.]
 [4. 4. 4.]
 [6. 6. 6.]]


In [38]:
dist_sq_matrix(x4, pbc={0: 50, 1: 50, 2: 50})

[[[ 0. -3. -5.]
  [ 3.  0. -2.]
  [ 5.  2.  0.]]

 [[ 0. -3. -5.]
  [ 3.  0. -2.]
  [ 5.  2.  0.]]

 [[ 0. -3. -5.]
  [ 3.  0. -2.]
  [ 5.  2.  0.]]]


array([[ 0., 27., 75.],
       [27.,  0., 12.],
       [75., 12.,  0.]])

In [29]:
%%timeit 
_ = dist_sq_matrix(x4, pbc={0: 50, 1: 50, 2: 50})

20.1 us +- 251 ns per loop (mean +- std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
bonds = bug.positions - np.take(bug.positions, range(1,201), axis=0, mode='wrap')
bond_lengths = np.linalg.norm(bonds, axis=1).reshape(bonds.shape[0],1)
bonds = np.divide(bonds, bond_lengths)
cosines = np.einsum('ij,ij->i', bonds[:-1], bonds[1:])

In [ ]:
plt.hist(bond_lengths)

In [ ]:
bond_lengths.mean()

In [ ]:
bonds.shape

In [ ]:
x1 = 2* np.ones(3)
print(x1)
x2 = np.ones(3)
print(x2)
x3 = np.array([x1, x2])#.reshape(2,3,1)
print(x3)
x4 = np.array([x1, x2])#.reshape(2,3,1)
print(x4)
np.dot(x3,x4.T)

In [ ]:
%%time
# defining collectors
# -bug:
gyr_t = []
principal_axes_t = np.empty([0, 3, 3])
asphericity_t = []
shape_parameter_t = []
sliced_trj = cell.trajectory[0: -1]
for _ in sliced_trj:
    # bug:
    # -various measures of chain size
    gyr_t.append(bug.radius_of_gyration())
    # -shape parameters:
    asphericity_t.append(bug.asphericity(pbc=False, unwrap=False))
    principal_axes_t = np.append(
        principal_axes_t,
        np.array([bug.principal_axes(pbc=False)]),
        axis=0
    )
    shape_parameter_t.append(bug.shape_parameter(pbc=False))
# Saving collectors to memory
# -bug
np.save(save_to + sim_name + '-gyrTMon.npy', np.array(gyr_t))
np.save(save_to + sim_name + '-asphericityTMon.npy',
        np.array(asphericity_t)
        )
np.save(save_to + sim_name + '-principalTMon.npy', principal_axes_t)
np.save(save_to + sim_name + '-shapeTMon.npy', np.array(shape_parameter_t))
# Simulation stamps:
outfile = save_to + sim_name + "-stamps.csv"
stamps_report(outfile, sim_info, n_frames)
print('done.')

In [ ]:
for (nuc_topo, nuc_trj) in nuc_pairs:
    prober.hns_nucleoid_cub(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

In [ ]:
gyr = np.loadtxt(gyr_file)
#gyr[:,1] = (gyr[:,1] - gyr[:,1].mean())/ np.var(gyr[:,1])
t_step=1
gyr = gyr[::t_step,:]
#t_i=2.13*10**8
#t_f=2.53*10**8
gyr = pd.DataFrame(gyr, columns=["t", "gyr"])
#gyr = gyr[gyr['t']<=1.03*10**8]
#gyr = gyr.loc[(gyr['t']>=t_i)&(gyr['t']<=t_f),:]
gyr['log_t'] = np.log10(gyr.t)
gyr_mean = gyr['gyr'].mean()
gyr.t.max()

In [ ]:
gyr = np.load('./N200epshm29nh37ac2nc5969l25dt0.005ndump2000adump5000ens1.ring-nucleoid-gyrTMon.npy')

In [ ]:
sim_real_dt

In [ ]:
gyr = pd.DataFrame(gyr, columns=["gyr"])
#gyr = gyr[gyr['t']<=1.03*10**8]
#gyr = gyr.loc[(gyr['t']>=t_i)&(gyr['t']<=t_f),:]
gyr['t'] = gyr.index * sim_real_dt
gyr['log_t'] = np.log10(gyr.t)
gyr_mean = gyr['gyr'].mean()
gyr.t.max()

### slicing over different

In [ ]:

t_choice = [  # upper 1.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.03*10**8, 1.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [0.53*10**8, 1.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [0.83*10**8, 1.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [0.93*10**8, 1.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [0.98*10**8, 1.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.02*10**8, 1.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 2.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [1.03*10**8, 2.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.53*10**8, 2.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.83*10**8, 2.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.93*10**8, 2.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.98*10**8, 2.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [2.02*10**8, 2.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 1.53*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.53*10**8, 1.53*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.03*10**8, 1.53*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.33*10**8, 1.53*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.43*10**8, 1.53*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.48*10**8, 1.53*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.52*10**8, 1.53*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
t_choice = [  # upper 1.53*10**8
    [0, 10*10**5, r'$\Delta t/\tau=25\times 10^7$'],
    [5*10**5, 10*10**5, r'$\Delta t/\tau=10\times 10^7$'],
    [1*10**5, 5*10**5, r'$\Delta t/\tau=5\times 10^7$'],
    [5*10**4, 10*10**4, r'$\Delta t/\tau=2\times 10^7$'],
    [2.5*10**4, 5*10**4, r'$\Delta t/\tau=1\times 10^7$'],
    [1.25*10**4, 2.5*10**4, r'$\Delta t/\tau=0.5\times 10^7$'],
    [0, 10**5, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
fig, axes = plt.subplots(7,1, figsize=(16,36))
fig.suptitle(r"$t_{final}/\tau=15.3\times 10^7$",y=0.98)
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
for ax, (t_i, t_f, dt) in zip(axes,t_choice):
    gyr_sliced = gyr.loc[(gyr['t']>=t_i)&(gyr['t']<=t_f),:]
    gyr_mean = np.round(gyr_sliced['gyr'].mean(),2)
    ax = sns.lineplot(
        data=gyr_sliced,
        x='t',
        y='gyr',
        ci=None,
        palette=color_palette,
        ax=ax,
        label=dt,
    )
    ax.axhline(
        gyr_mean, color='royalblue',
        label=rf'$\langle \sqrt{{R^2_g}}\rangle={gyr_mean}$', 
        lw=2, ls='-'
    )
    ax.legend(facecolor='white', loc='upper left', ncol=2)
    ax.set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
    ax.set_xlabel(r'$t/\tau$')
fig.tight_layout(w_pad=1, h_pad=0)
plt.show()
fig.savefig("gyrTDna-slices.pdf")

### Semi-log to show reaching equilibrium

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16,18))
color_palette = 'flare'
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
axes[0] = sns.lineplot(
    data=gyr,
    x='t',
    y='gyr',
    ci=None,
    palette=color_palette,
    ax=axes[0]
)
fig.suptitle(r"$\Delta t_{sim}/\tau=28.7\times 10^7$",y=0.92)
axes[0].set_xlabel(r'$t/\tau$')
axes[0].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$', 
                lw=2, ls='-')
axes[0].legend(facecolor='white', loc='upper right')
axes[0].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
axes[1].semilogx(
    gyr['t'],
    gyr['gyr']
)
axes[1].set_xlabel(r'$\log(t/\tau)$')
axes[1].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$', 
                lw=2, ls='-')
axes[1].legend(facecolor='white', loc='upper right')
axes[1].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
plt.show()
fig.savefig("gyrTDna-semiLogX.pdf")